# Pandas Data Wrangling

### Homework

Before you begin, remember to import the necessary libraries.

In [2]:
import numpy as np
import pandas as pd

#### Standard Exercises

1. Load the `lifeexpectancy.csv` and the `drinks.csv` datasets from the `data` > `Life Expectancy` folder and save them, respectively, to the following two objects: `life_raw` and `drinks_raw`.

In [3]:
life_raw = pd.read_csv("data/Life expectancy/lifeexpectancy.csv")

In [4]:
drinks_raw = pd.read_csv("data/Life Expectancy/drinks.csv")

2. For each DataFrame, create a copy and save it to a new object (`life` and `drinks`), check its columns data type and make sure each column is in the expected format and whether there are any missing values. *(hint: remember the `.info()` method)* 

In [5]:
life = life_raw.copy()
drinks = drinks_raw.copy()

In [6]:
print(life.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6408 entries, 0 to 6407
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   GhoCode                        6408 non-null   object 
 1   GhoDisplay                     6408 non-null   object 
 2   PublishStateCode               6408 non-null   object 
 3   PublishStateDisplay            6408 non-null   object 
 4   YearCode                       6408 non-null   int64  
 5   YearDisplay                    6408 non-null   int64  
 6   RegionCode                     6408 non-null   object 
 7   RegionDisplay                  6408 non-null   object 
 8   WorldBankIncomeGroupGroupCode  5244 non-null   object 
 9   WorldBankIncomeGroupDisplay    5244 non-null   object 
 10  CountryCode                    6408 non-null   object 
 11  CountryDisplay                 6408 non-null   object 
 12  SexCode                        6408 non-null   o

In [7]:
print(drinks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    object 
 2   spirit_servings               193 non-null    object 
 3   wine_servings                 193 non-null    object 
 4   total_litres_of_pure_alcohol  0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 7.7+ KB
None


3. The `drinks` DataFrame has three columns (the ones ending with `_servings`) that should be interpreted as integer but are in fact stored as strings (ie: object). Find out why that is happening and replace any erroneous character with a zero. Then, convert each of those three columns to integers. *(hint: a quick way to check the contents of a column is to use the `.unique()` method)*

In [8]:
drinks["beer_servings"] = pd.to_numeric(drinks["beer_servings"], errors="coerce")

In [9]:
drinks["wine_servings"] = pd.to_numeric(drinks["wine_servings"], errors="coerce")

In [10]:
drinks["spirit_servings"] = pd.to_numeric(drinks["spirit_servings"], errors="coerce")

In [11]:
country_region = life[["CountryDisplay", "RegionDisplay"]].drop_duplicates()

4. The `.drop_duplicates()` method allows you to remove duplicated rows from a DataFrame (read [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html) to find out more). Select the `CountryDisplay` and `RegionDisplay` columns from the `life` DataFrame and remove the duplicated rows. Save the final output in a new dataset called `country_region`. You just created a lookup-table that should have 195 rows and 2 columns. 

In [12]:
print(country_region.shape)

(195, 2)


5. Using the `.merge()` method, combine the and the `drinks` and the `country_region` datasets using a **left join** in order to **add the `RegionDisplay` field** to the `drinks` DataFrame. Call the new DataFrame object `drinks_region`.

In [13]:
# country_region.head()
drinks_region = drinks.merge(country_region, how="left", left_on="country", right_on="CountryDisplay")

6. Rename the `RegionDisplay` column to `region` and reorder the columns in the `drinks_region` DataFrame so to have the following sequence: ['region', 'country', 'beer_servings', 'spirit_servings', 'wine_servings'].

In [27]:
drinks_region.rename(columns={"RegionDisplay": "region"}, inplace=True)
print(drinks_region.columns)

Index(['country', 'beer_servings', 'spirit_servings', 'wine_servings',
       'total_litres_of_pure_alcohol', 'CountryDisplay', 'region'],
      dtype='object')


In [32]:
drinks_new = drinks_region[['region', 'country', 'beer_servings', 'spirit_servings', 'wine_servings']]
drinks_new

,region,country,beer_servings,spirit_servings,wine_servings
0,Eastern Mediterranean,Afghanistan,0.0,0.0,0.0
1,Europe,Albania,89.0,132.0,54.0
2,Africa,Algeria,25.0,0.0,14.0
3,Europe,Andorra,245.0,138.0,312.0
4,Africa,Angola,217.0,57.0,45.0
...,...,...,...,...,...
188,NaN,Venezuela,333.0,100.0,3.0
189,NaN,Vietnam,111.0,2.0,1.0
190,Eastern Mediterranean,Yemen,6.0,0.0,0.0
191,Africa,Zambia,32.0,19.0,4.0


7. Find out which `region` has the **highest average spirit_servings**.

In [40]:
drinks_new.groupby(by="region").agg({"spirit_servings": "mean"}).sort_values(by="spirit_servings", ascending=False)

,spirit_servings
region,
Americas,143.148148
Europe,130.812500
Western Pacific,78.863636
South_East Asia,53.777778
Eastern Mediterranean,21.578947
Africa,17.487805


#### Advanced Exercise

1. Using the `.merge()` method, combine the `life` and the `drinks` datasets using a **left join** on the appropriate field. Save the new dataset in a new DataFrame object called `life_drinks_raw`.

2. Create a copy of `life_drinks_raw` called `life_drinks` where you filter the above DataFrame using the following conditions (check out the solution on [this Stack Overflow page](https://stackoverflow.com/questions/22546425/how-to-implement-a-boolean-search-with-multiple-columns-in-pandas) to get a cue on how to filter data on multiple columns in a pandas DataFrame): 

- `GhoDisplay` = 'Life expectancy at birth (years)'
- `WorldBankIncomeGroupDisplay` = null *(hint: remember the isnull() method)*
- `SexDisplay` != 'Both sexes'

3. Reorder the columns in the `life_drinks` DataFrame so to have the following sequence: 
['GhoDisplay', 'YearDisplay', 'YearDisplay', 'RegionDisplay', 'CountryDisplay', 'SexDisplay', 'Numeric', 'beer_servings',
       'spirit_servings', 'wine_servings']

4. Rename the following columns as specified: 

- Numeric --> LifeExpectancy
- beer_servings = BeerServings
- spirit_servings = SpiritServings
- wine_servings = WineServings

5. Create a new DataFrame, call it `life_drinks_13`, where you show the average `LifeExpectancy` and `SpiritServings` by `RegionDisplay` in 2013. Can you see a relationship between the two metrics? If so, would you trust this data to make any broad conclusions?

*(Hint: remember to include the `as_index=False` parameter in the groupby so to aviod multi-index DataFrames)*

6. Now create a new DataFrame, call it `life_exp_13`, where you show the average `LifeExpectancy` by `RegionDisplay` and `SexDisplay` in 2013.

7. Starting from the `life_exp_13` DataFrame and using an **inner join**, create a DataFrame named `life_exp_13_gndr` that shows one row per `RegionDisplay` (6 rows) and two columns, each containing the average male and female life expectancy (relative to each region). 

    - Then rename and reorder the DataFrame's columns so to end up with the following column names: ['RegionDisplay', 'LifeExpMale', 'LifeExpFemale']. 
    - Which `DisplayRegion` has the highest gap in years between males and females?